In [42]:
import pandas as pd
import numpy as np
import os
filepath = os.path.abspath('')
samplesFilepath = filepath + '/data-from-samples.csv'
calibrationFilepath = filepath + '/calibration-data.csv'
MuNaught = float('1.257e-6')
beta = 0.42
g=9.81
volumeContainer = 4.395

In [43]:
def chaiEffectiveEquation(vSample,chai,vAir):
    return (vSample) * chai / (vSample + vAir)
def calculateVolume(mass,density):
    return mass / density

In [44]:
samplesData = pd.read_csv(samplesFilepath)
calibrationData = pd.read_csv(calibrationFilepath)
display(calibrationData)

,Current,Voltage,High,Low
0,1.50,0.4,-0.643,0.449
1,0.00,0.0,0.003,NaN
2,-1.50,0.4,0.652,-0.533
3,1.25,0.4,-0.537,0.336
4,-1.25,0.3,0.549,-0.382
5,1.75,0.4,-0.756,0.487
6,-1.75,0.4,0.764,-0.447
7,2.00,0.5,-0.865,0.444
8,-2.00,0.5,0.877,-0.486


In [45]:
class Sample:
    def __init__(self,name,high,low,state):
        self.name = name
        self.high = high * float('1e-3')
        self.low = low * float('1e-3')
        self.far = 0
        self.state = state
        self.chiLow = (2* self.low * MuNaught *g) /((0.0095**2)*(beta**2))
        self.chiHigh = (2* self.high * MuNaught *g) /((0.0095**2)*(-beta**2))

In [46]:
samples = {}
listChiLow = []
listChiHigh = []
listChaiEffective = []
listVolume = []
for i in samplesData.index:
    name = samplesData['Sample'][i]
    high = samplesData['High'][i]
    low = samplesData['Low'][i]
    state = samplesData['State'][i]
    samples[name] = Sample(name,high,low,state)
    if state == 'solid' and  0 == 1:
        
        area = float(samplesData['width(mm)'][i])/100 * float(samplesData['depth(mm)'][i])/100
        samples[name].chiLow = (2* samples[name].low * MuNaught *g) /((area**2)*(beta**2))
        samples[name].chiHigh = (2* samples[name].high * MuNaught *g) /((area**2)*(beta**2))
        
        listChiLow.append(samples[name].chiLow)
        listChiHigh.append(samples[name].chiHigh)
    listChiLow.append(samples[name].chiLow)
    listChiHigh.append(samples[name].chiHigh)
    if state == 'powder':
        density = float(samplesData['Density'][i])
        mass = float(samplesData['Mass'][i])
        vSample = calculateVolume(mass,density)
        vAir = volumeContainer - vSample
        chaiEffective = chaiEffectiveEquation(vSample,samples[name].chiHigh,vAir)
        listChaiEffective.append(chaiEffective)
        samples[name].chaiEffective = chaiEffective
        listVolume.append(vSample)
    elif state == 'powderMix1':
        tMass = float(samplesData['Mass'][i])
        massPercentages = [0.55,64.91,34.54]
        densityList = [5.37,3.95,0.96]
        vSample = 0
        for i in range(len(massPercentages)):
            mass = massPercentages[i] / 100 * tMass
            density = densityList[i]
            vSample += calculateVolume(mass,density)
        chaiEffective = chaiEffectiveEquation(vSample,samples[name].chiHigh,vAir)
        listChaiEffective.append(chaiEffective)
        listVolume.append(vSample)
    elif state == 'solid':
        area = float(samplesData['width(mm)'][i])/100 * float(samplesData['depth(mm)'][i])/100
        listChaiEffective.append((2* samples[name].high * MuNaught *g) /((area**2)*(beta**2)))
        listVolume.append('NaN')
    else:
        listChaiEffective.append('NaN')
        listVolume.append('NaN')



In [47]:
samplesData['Chi Low'] = listChiLow
samplesData['Chi High'] = listChiHigh
samplesData['Volume'] = listVolume
samplesData['Chi Effective'] = listChaiEffective
display(samplesData)

,SampleNumber,Sample,Far,High,Low,length(mm),width(mm),depth(mm),State,Mass,Density,Chi Low,Chi High,Volume,Chi Effective
0,1,copper,0,0.010,-0.008,32.48,9.47,9.48,solid,NaN,NaN,-0.000012,-0.000015,NaN,0.000017
1,2,aluminum,0,-0.009,0.015,31.92,9.47,9.40,solid,NaN,NaN,0.000023,0.000014,NaN,-0.000016
2,3,titanium,0,-0.131,0.133,32.86,9.55,9.50,solid,NaN,NaN,0.000206,0.000203,NaN,-0.000223
3,4,bismuth,0,0.066,-0.070,NaN,NaN,NaN,powder,15.412,9.780,-0.000108,-0.000102,1.575869,-0.000037
4,5,cobal wire,0,-0.198,0.211,NaN,NaN,NaN,Na,NaN,NaN,0.000327,0.000307,NaN,NaN
5,6,pyrolytic graphite,0,0.098,-0.096,NaN,NaN,NaN,Na,NaN,NaN,-0.000149,-0.000152,NaN,NaN
6,7,hexahydrate,0,-0.243,0.241,NaN,NaN,NaN,powder,4.841,4.139,0.000373,0.000376,1.169606,0.0001
7,8,gadolinium oxide,0,-2.573,2.576,NaN,NaN,NaN,powder,6.301,7.410,0.003991,0.003986,0.850337,0.000771
8,9,erbium oxide,0,-5.268,5.072,NaN,NaN,NaN,powder,8.513,8.640,0.007857,0.008161,0.985301,0.00183
9,10,Mohr's salt,0,-0.370,0.370,NaN,NaN,NaN,powder,3.487,1.860,0.000573,0.000573,1.874731,0.000244


In [51]:
lengthX = 9.9 * float('1e-3')
class CalibrateData:
    def __init__(self,current,voltage,high,low):
        self.current = current
        self.voltage = voltage
        self.high = high
        self.low = low
        self.betaYHigh = ((abs(high) * float('1e-3')) * g) / (lengthX * current)
        self.betaYLow = ((abs(low) * float('1e-3')) * g) / (lengthX * current)

In [52]:
calibrates = {}
listBetaYLow = []
listBetaYHigh = []
for i in calibrationData.index:
    current  = calibrationData['Current'][i]
    voltage = calibrationData['Voltage'][i]
    high = calibrationData['High'][i]
    low = calibrationData['Low'][i]
    if current:
        calibrates[current] = CalibrateData(current,voltage,high,low)
        listBetaYHigh.append(calibrates[current].betaYHigh)
        listBetaYLow.append(calibrates[current].betaYLow)
    else:
        listBetaYHigh.append('NaN')
        listBetaYLow.append('Nan')


In [53]:
calibrationData['B_y-high'] = listBetaYHigh
calibrationData['B_y-low'] = listBetaYLow
display(calibrationData)

,Current,Voltage,High,Low,B_y-high,B_y-low
0,1.50,0.4,-0.643,0.449,0.42477,0.296612
1,0.00,0.0,0.003,NaN,NaN,Nan
2,-1.50,0.4,0.652,-0.533,-0.430715,-0.352103
3,1.25,0.4,-0.537,0.336,0.425695,0.266356
4,-1.25,0.3,0.549,-0.382,-0.435207,-0.302822
5,1.75,0.4,-0.756,0.487,0.428073,0.275756
6,-1.75,0.4,0.764,-0.447,-0.432603,-0.253106
7,2.00,0.5,-0.865,0.444,0.428568,0.219982
8,-2.00,0.5,0.877,-0.486,-0.434514,-0.240791
